In [29]:
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import numpy as np

In [3]:
dataset_path = os.path.join(Path.cwd().parent, 'data', 'stackexchange_dataset.csv')
data = pd.read_csv(dataset_path,index_col='question_id')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 79802517 to 17945108
Data columns (total 11 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   title                          100000 non-null  object 
 1   has_accepted_answer            100000 non-null  bool   
 2   accepted_answer_score          12000 non-null   float64
 3   time_to_accepted_answer_hours  12000 non-null   float64
 4   question_score                 100000 non-null  int64  
 5   question_text                  100000 non-null  object 
 6   num_tags                       100000 non-null  int64  
 7   tags                           100000 non-null  object 
 8   accepted_answer_id             12000 non-null   float64
 9   accepted_answer_length_chars   12000 non-null   float64
 10  accepted_answer_length_tokens  12000 non-null   float64
dtypes: bool(1), float64(5), int64(2), object(3)
memory usage: 8.5+ MB


In [5]:
data.head()

,title,has_accepted_answer,accepted_answer_score,time_to_accepted_answer_hours,question_score,question_text,num_tags,tags,accepted_answer_id,accepted_answer_length_chars,accepted_answer_length_tokens
question_id,,,,,,,,,,,
79802517,Looking for a better way using &quot;.Include&...,False,NaN,NaN,2,I am looking for a better way to use the .Incl...,2,"['c#', 'entity-framework']",NaN,NaN,NaN
79802934,NTP is moving my clock further from the correc...,False,NaN,NaN,0,Windows 11 Pro 10.0.26200 Build 26200. Dell XP...,1,['ntp'],NaN,NaN,NaN
79802909,Execution of pandas&#39; info in python,False,NaN,NaN,0,I am new to pandas library in python. When I l...,2,"['python', 'pandas']",NaN,NaN,NaN
79802932,How to debug MongoDB recurring error 314 Objec...,False,NaN,NaN,0,On a fresh Mongo DB 8.0 mono instance (no clus...,4,"['database', 'mongodb', 'nosql', 'system-admin...",NaN,NaN,NaN
79802919,How to integrate QML UI into a custom Vulkan r...,False,NaN,NaN,0,I'm developing a custom Vulkan renderer and wa...,4,"['c++', 'qt', 'qml', 'vulkan']",NaN,NaN,NaN


In [6]:
data.index.is_unique

False

In [7]:
data[data.index.duplicated()].shape[0]

8

In [8]:
data[data.index.duplicated(keep=False)].head(4)

,title,has_accepted_answer,accepted_answer_score,time_to_accepted_answer_hours,question_score,question_text,num_tags,tags,accepted_answer_id,accepted_answer_length_chars,accepted_answer_length_tokens
question_id,,,,,,,,,,,
79802647,Paypal webhook simulator not firing,False,NaN,NaN,0,To test a PayPal integration that also reacts ...,2,"['paypal', 'webhooks']",NaN,NaN,NaN
79802647,Paypal webhook simulator not firing,False,NaN,NaN,0,To test a PayPal integration that also reacts ...,2,"['paypal', 'webhooks']",NaN,NaN,NaN
79789014,Can I use SSM to run command for SQL query exe...,False,NaN,NaN,-1,I want to automate a database drop and databas...,3,"['sql-server', 'amazon-web-services', 'aws-ssm']",NaN,NaN,NaN
79789014,Can I use SSM to run command for SQL query exe...,False,NaN,NaN,-1,I want to automate a database drop and databas...,3,"['sql-server', 'amazon-web-services', 'aws-ssm']",NaN,NaN,NaN


Since they are the same question, we drop them

In [9]:
data2 = data[~data.index.duplicated()]

In [10]:
data2.shape,data.shape

((99992, 11), (100000, 11))

In [11]:
data2.index.is_unique

True

---

In [12]:
data2['question_score'].max(),data2['question_score'].min()

(np.int64(27487), np.int64(-20))

In [13]:
data2['has_accepted_answer'].where(data2['has_accepted_answer']==True).dropna().count(),data2['has_accepted_answer'].where(data2['has_accepted_answer']==False).dropna().count()

(np.int64(39938), np.int64(60054))

Huge disproportion in the data in regards to accepted answer

In [14]:
data_acc = data2[data2['has_accepted_answer']]

In [15]:
data_acc.shape

(39938, 11)

In [16]:
data_acc[data_acc['time_to_accepted_answer_hours'].notna()].shape

(12000, 11)

Out of those 39938 questions, only 12000 has accepted answer

In [17]:
data_aac = data_acc[data_acc['time_to_accepted_answer_hours'].notna()]
data_aac.shape

(12000, 11)

In [19]:
data_aac

,title,has_accepted_answer,accepted_answer_score,time_to_accepted_answer_hours,question_score,question_text,num_tags,tags,accepted_answer_id,accepted_answer_length_chars,accepted_answer_length_tokens
question_id,,,,,,,,,,,
1562367,How do short URLs services work?,True,242.0,0.499167,119,How do services like TinyURL or Metamark work?...,5,"['web-services', 'url', 'url-shortener', 'bit....",1562539.0,3442.0,409.0
79802633,How to import based on variable to avoid loadi...,True,1.0,2.591667,0,Using React Native+Expo I'm trying to load onl...,5,"['javascript', 'reactjs', 'node.js', 'typescri...",79802815.0,1290.0,152.0
79801576,Is std::unique_ptr needed on abstract class wh...,True,1.0,10.045833,1,I came across below code in a project I am wor...,5,"['c++', 'c++20', 'unique-ptr', 'std-function',...",79802123.0,3496.0,426.0
46853200,How to make zsh&#39;s REPORTTIME work? (time f...,True,8.0,2.211667,6,This is my .zshrc: export REPORTTIME=3 When I ...,1,['zsh'],46855194.0,1945.0,245.0
2711001,How to apply shell command to each line of a c...,True,347.0,0.055000,328,Suppose I have some output from a command (suc...,1,['bash'],2711011.0,433.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...
25045932,Fake DbFunctions DiffDays,True,0.0,6541.563889,1,I am trying to fake DbFunctions.DiffDays but a...,2,"['c#', 'unit-testing']",29939853.0,277.0,28.0
71355883,Next-Auth Signout within jwt callback,True,20.0,481.200556,10,If a user is signed into the app between app u...,4,"['javascript', 'reactjs', 'next.js', 'next-auth']",71608606.0,1410.0,185.0
78467875,freeing allocated memory returned from functio...,True,-1.0,0.244444,-1,If I have a function like this: char* f1(char ...,3,"['c++', 'memory-management', 'memory-leaks']",78467920.0,1138.0,134.0


---
## Tags

In [ ]:
import ast

In [65]:
tags=data_aac[['tags','num_tags']]
tags.loc[:,('tags')] = tags.loc[:,('tags')].apply(ast.literal_eval)


In [66]:
tags.iloc[0]['tags']

['web-services', 'url', 'url-shortener', 'bit.ly', 'short-url']

In [75]:
tag_counter = Counter(tag for tags in tags['tags'] for tag in tags)
tag_freq = pd.Series(tag_counter).sort_values(ascending=False)

In [76]:
tag_freq

python             1528
c#                  746
javascript          703
c++                 689
java                592
                   ... 
gocql                 1
screen                1
b-tree-index          1
beeware               1
outlook-restapi       1
Length: 7684, dtype: int64

### Problem: Extreme Multi-label Classification

As seen above, there are $7684$ unique tags. This is too big for a classification problem.

We can use:
- Frequency-Based Filtering
    - Keep only the top `N` most frequent tags
- "Semantic Compression" using Embeddings and Clustering
  - Use embeddings and clustering to group them

